In [1]:
import pyproj
from pyproj import Geod
pyproj.__version__

'3.3.1'

In [2]:
def dlon_at_lat(lat, d, g):
    """
    纬度lat处，大地距离d，对应的经度变化
    不同纬度上，距离d对应的经度变化不同
    """
    new_lon, new_lat, az = g.fwd(0, lat, 90, d)
    dlon = new_lon - 0
    return dlon

def dlat_at_lat(d ,g):
    """
    大地距离d对应的纬度变化
    经线是测地线，距离d对应的纬度即变化纬度
    """
    lat = 0
    new_lon, new_lat, az = g.fwd(0, lat, 0, d)
    dlat = new_lat - lat
    return dlat

def grid(lat, gdlat):
    return round(lat/gdlat)

def bbox(clat, clon, r, gdlat, gdlon, g):
    cgrida = grid(clat, gdlat) # 网格点号
    cgrido = grid(clon, gdlon)
    clon, clat = cgrido * gdlon, cgrida * gdlat # 格点中心经纬度

    elon, wlon = clon + gdlon * 0.5, clon - gdlon * 0.5 # 格点四边经纬度
    nlat, slat = clat + gdlat * 0.5, clat - gdlat * 0.5

    ndlat = dlat_at_lat(r, g) # 四角处向外扩展经纬度
    sdlat = dlat_at_lat(r, g)
    ndlon = dlon_at_lat(nlat, r, g)
    sdlon = dlon_at_lat(slat, r, g)

    nnlat = nlat + ndlat # 四角点外扩后的经纬度
    sslat = slat - sdlat 
    eelon = max( elon + ndlon, elon + sdlon )
    wwlon = min( wlon - ndlon, wlon - sdlon )

    return grid(nnlat, gdlat), grid(eelon, gdlon), grid(sslat, gdlat), grid(wwlon, gdlon)

def box(lat, lon, r, g):
    dlat = dlat_at_lat(r, g)
    dlon = dlon_at_lat(lat, r, g)
    return lat + dlat, lon + dlon, lat - dlat, lon - dlon


def index_table(gdlat, gdlon, r, g):
    nesw = [ bbox(l, 0, r, gdlat, gdlon, g) for  l in [ gdlat * i for i in range(round(-90/gdlat)+1 , round(90/gdlat))]]
    e = [ e for (n, e, s, w) in nesw]
    return [(round(-90/gdlat), round(180/gdlon))] + [(i, e) for (i,e) in zip(range(round(-90/gdlat)+1 , round(90/gdlat)), e)] + [(round(90/gdlat), round(180/gdlon))]

In [3]:
print(box(0,0,7700, Geod(ellps='WGS84')))
print(box(90,0,7700, Geod(ellps='WGS84')))
print(box(-90,0,7700, Geod(ellps='WGS84')))
print(box(90,180,7700, Geod(ellps='WGS84')))
print(box(90,-180,7700, Geod(ellps='WGS84')))
print(box(-90,180,7700, Geod(ellps='WGS84')))
print(box(-90,-180,7700, Geod(ellps='WGS84')))

(0.06963644938857323, 0.06917027687720315, -0.06963644938857323, -0.06917027687720315)
(90.06963644938857, 90.0, 89.93036355061143, -90.0)
(-89.93036355061143, 90.0, -90.06963644938857, -90.0)
(90.06963644938857, 270.0, 89.93036355061143, 90.0)
(90.06963644938857, -90.0, 89.93036355061143, -270.0)
(-89.93036355061143, 270.0, -90.06963644938857, 90.0)
(-89.93036355061143, -90.0, -90.06963644938857, -270.0)


In [5]:
gdlat = gdlon = 1.0 # 经纬度划分步长
r = 7.7 * 1000.0 # 景角点到中心点距离
g = Geod(ellps='WGS84')

In [6]:
dlat_at_lat(1000, g)

0.009043694769749644

In [14]:
import pandas as pd 
import numpy as np
index01 = pd.DataFrame([(lat, dlon_at_lat(lat, 1000, g)) for lat in np.linspace(0, 90, 361) ])
index01.to_csv(f"lon_index.csv")


In [16]:
def dlat_at_lat(lat, d ,g):
    """
    大地距离d对应的纬度变化
    经线是测地线，距离d对应的纬度即变化纬度
    """
    new_lon, new_lat, az = g.fwd(0, lat, 0, d)
    dlat = new_lat - lat
    return dlat

index01 = pd.DataFrame([(lat, dlat_at_lat(lat, 1000, g)) for lat in np.linspace(0, 90, 361) ])
index01.to_csv(f"lon_index_by_lat.csv")
index01

,0,1
0,0.00,0.009044
1,0.25,0.009044
2,0.50,0.009044
3,0.75,0.009044
4,1.00,0.009044
...,...,...
356,89.00,0.008953
357,89.25,0.008953
358,89.50,0.008953
359,89.75,0.008953


In [11]:
index01

,0,1
0,0.000000,0.008983
1,0.250696,0.008983
2,0.501393,0.008983
3,0.752089,0.008984
4,1.002786,0.008985
...,...,...
355,88.997214,0.511559
356,89.247911,0.682049
357,89.498607,1.022997
358,89.749304,2.045322


In [14]:
import pandas as pd
gdlat = gdlon = 1.0 # 经纬度划分步长
r = 7.7 * 1000.0 # 景角点到中心点距离
g = Geod(ellps='WGS84')
index01 = pd.DataFrame([(l, e) for (l, e) in index_table(gdlat, gdlon, r, g)],
    columns=["index", "halfwid"]
)
index02 = pd.DataFrame([(l, e) for (l, e) in index_table(gdlat*0.5, gdlon*0.5, r, g)],
    columns=["index", "halfwid"]
)
index04 = pd.DataFrame([(l, e) for (l, e) in index_table(gdlat*0.25, gdlon*0.25, r, g)],
    columns=["index", "halfwid"]
)
index08 = pd.DataFrame([(l, e) for (l, e) in index_table(gdlat/8, gdlon/8, r, g)],
    columns=["index", "halfwid"]
)
index16 = pd.DataFrame([(l, e) for (l, e) in index_table(gdlat/16, gdlon/16, r, g)],
    columns=["index", "halfwid"]
)
index32 = pd.DataFrame([(l, e) for (l, e) in index_table(gdlat/32, gdlon/32, r, g)],
    columns=["index", "halfwid"]
)
index64 = pd.DataFrame([(l, e) for (l, e) in index_table(gdlat/64, gdlon/64, r, g)],
    columns=["index", "halfwid"]
)
index128 = pd.DataFrame([(l, e) for (l, e) in index_table(gdlat/128, gdlon/128, r, g)],
    columns=["index", "halfwid"]
)

In [26]:
import lux
index01

,index,halfwid
0,-90,180
1,-89,8
2,-88,3
3,-87,2
4,-86,2
...,...,...
176,86,2
177,87,2
178,88,3
179,89,8


In [8]:
index01.to_excel("index01.xlsx")

In [17]:
dfs = [index01, index02, index04, index08, index16, index32, index64, index128]
for i in range(0, 8):
    d = 2**i 
    dfs[i].to_excel(f"halfwid-index-{d}.xlsx")
    dfs[i].to_csv(f"halfwid-index-{d}.csv")


In [2]:
import random
help(random.random)

Help on built-in function random:

random() method of random.Random instance
    random() -> x in the interval [0, 1).



In [3]:
help(random.randint)

Help on method randint in module random:

randint(a, b) method of random.Random instance
    Return random integer in range [a, b], including both end points.



In [4]:
dir(random)

['BPF',
 'LOG4',
 'NV_MAGICCONST',
 'RECIP_BPF',
 'Random',
 'SG_MAGICCONST',
 'SystemRandom',
 'TWOPI',
 '_Sequence',
 '_Set',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_accumulate',
 '_acos',
 '_bisect',
 '_ceil',
 '_cos',
 '_e',
 '_exp',
 '_inst',
 '_log',
 '_os',
 '_pi',
 '_random',
 '_repeat',
 '_sha512',
 '_sin',
 '_sqrt',
 '_test',
 '_test_generator',
 '_urandom',
 '_warn',
 'betavariate',
 'choice',
 'choices',
 'expovariate',
 'gammavariate',
 'gauss',
 'getrandbits',
 'getstate',
 'lognormvariate',
 'normalvariate',
 'paretovariate',
 'randint',
 'random',
 'randrange',
 'sample',
 'seed',
 'setstate',
 'shuffle',
 'triangular',
 'uniform',
 'vonmisesvariate',
 'weibullvariate']

BPF
LOG4
NV_MAGICCONST
RECIP_BPF
Random
SG_MAGICCONST
SystemRandom
TWOPI
_Sequence
_Set
__all__
__builtins__
__cached__
__doc__
__file__
__loader__
__name__
__package__
__spec__
_accumulate
_acos
_bisect
_ceil
_cos
_e
_exp
_inst
_log
_os
_pi
_random
_repeat
_sha512
_sin
_sqrt
_test
_test_generator
_urandom
_warn
betavariate
choice
choices
expovariate
gammavariate
gauss
getrandbits
getstate
lognormvariate
normalvariate
paretovariate
randint
random
randrange
sample
seed
setstate
shuffle
triangular
uniform
vonmisesvariate
weibullvariate
